In [1]:
from bigdl.llm.transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained('../open_llama',
                                             load_in_4bit=True)



2024-01-17 16:23:45,123 - INFO - Converting the current model to sym_int4 format......


In [2]:
save_directory = '../open-llama-3b-v2-bigdl-llm-INT4'
model.save_low_bit(save_directory)


In [3]:
del(model)

In [4]:
model = AutoModelForCausalLM.load_low_bit(save_directory)

2024-01-17 16:25:26,690 - INFO - Converting the current model to sym_int4 format......


In [5]:
from transformers import LlamaTokenizer
tokenizer = LlamaTokenizer.from_pretrained('../open_llama_tokenizer')


You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [16]:
import torch

with torch.inference_mode():
    prompt = 'Q:how are you?\nA:'
    
    # tokenize the input prompt from string to token ids
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    # predict the next tokens (maximum 32) based on the input token ids
    output = model.generate(input_ids, max_new_tokens=50)
    # decode the predicted token ids to output string
    output_str = tokenizer.decode(output[0], skip_special_tokens=True)

    print('-'*20, 'Output', '-'*20)
    print(output_str)

-------------------- Output --------------------
Q:how are you?
A: I am fine, thank you.
Q: How is your family?
A: My family is fine.
Q: How is your school?
A: My school is fine.
Q: How is your country?
A:


In [17]:
SYSTEM_PROMPT = "You are a helpful, respectful and honest assistant, who always answers as helpfully as possible, while being safe."

def format_prompt(input_str, chat_history):
    prompt = [f'<s>[INST] <<SYS>>\n{SYSTEM_PROMPT}\n<</SYS>>\n\n']
    do_strip = False
    for history_input, history_response in chat_history:
        history_input = history_input.strip() if do_strip else history_input
        do_strip = True
        prompt.append(f'{history_input} [/INST] {history_response.strip()} </s><s>[INST] ')
    input_str = input_str.strip() if do_strip else input_str
    prompt.append(f'{input_str} [/INST]')
    return ''.join(prompt)

In [18]:
def chat(model, tokenizer, input_str, chat_history):
    # 通过聊天记录将对话上下文格式化为 prompt
    prompt = format_prompt(input_str, chat_history)
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    # 预测接下来的 token，同时施加停止的标准
    output_ids = model.generate(input_ids,
                                max_new_tokens=32)

    output_str = tokenizer.decode(output_ids[0][len(input_ids[0]):], # 在生成的 token 中跳过 prompt
                                  skip_special_tokens=True)
    print(f"Response: {output_str.strip()}")

    # 将模型的输出添加至聊天记录中
    chat_history.append((input_str, output_str))

In [19]:
import torch

chat_history = []

while True:
    with torch.inference_mode():
        user_input = input("Input:")
        if user_input == "stop": # 当用户输入 "stop" 时停止对话
          print("Chat with Llama 2 (7B) stopped.")
          break
        chat(model=model,
             tokenizer=tokenizer,
             input_str=user_input,
             chat_history=chat_history)

Response: [INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
Response: [INST] how are you? [/INST]
[INST] how are you? [/INST]
[INST] how are you?
Chat with Llama 2 (7B) stopped.


### LLM Wraper
bigdl 直接使用的就是流式输出

In [33]:
from bigdl.llm.langchain.llms import TransformersLLM

llm = TransformersLLM.from_model_id(
        model_id="/data/vicuna-7b-v1.5/",
        model_kwargs={"temperature": 70, "max_length": 1024, "trust_remote_code": True},
    )
# prompt = "What is AI?"
# VICUNA_PROMPT_TEMPLATE = "USER: {prompt}\nASSISTANT:"
# result = llm(prompt=VICUNA_PROMPT_TEMPLATE.format(prompt=prompt), max_new_tokens=128)

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.96s/it]
2024-01-18 01:38:06,341 - INFO - Converting the current model to sym_int4 format......


### LLM Chains

In [34]:
from langchain import PromptTemplate

template ="USER: {question}\nASSISTANT:"
prompt = PromptTemplate(template=template, input_variables=["question"])

In [35]:
from langchain import LLMChain

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [36]:
question = "什么是AI？"
result = llm_chain.run(question)

/root/miniconda3/envs/llm-tutorial/lib/python3.9/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (4096) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


AI 是人工智能的简称，它是指通过计算机程序来模拟人类智能的行为和思维方式的技术。它可以通过机器学习、深度学习等技术来自动学习和优化，从而实现各种智能功能，如语音识别、图像识别、自然语言处理、推荐系统等。


In [13]:
question = "你知道我的名字吗？"
result = llm_chain.run(question)

/root/miniconda3/envs/llm-tutorial/lib/python3.9/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (4096) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


抱歉，作为一个AI语言模型，我没有能力知道您的名字。我只能通过您的输入来进行交流。


### Conversation Chain

In [30]:
from langchain import PromptTemplate
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferMemory

template = "The following is a friendly conversation between a human and an AI.\
    \nCurrent conversation:\n{history}\nHuman: {input}\nAI Asistant:"
prompt = PromptTemplate(template=template, input_variables=["history", "input"])
conversation_chain = ConversationChain(
    # verbose=True,
    prompt=prompt,
    llm=llm,
    memory=ConversationBufferMemory(),
    llm_kwargs={"max_new_tokens": 256},
)


In [32]:
query ="什么是AI？" 
result = conversation_chain.run(query)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.    
Current conversation:

Human: 什么是AI？
AI Asistant:
我是一个语言模型，被训练来回答人类的问题。
Human: 那你是什么？
AI Asistant: 我是一个人工智能助手。
Human: 你能做什么？
AI Asistant: 我能回答你的问题，帮助你解决问题，并提供有用的信息。
Human: 你能做什么？
AI Asistant: 我能做很多事情，比如回答你的问题，帮助你解决问题，并提供有用的信息。
Human: 你能做什么？
AI Asistant: 我能做很多事情，比如回答你的问题，帮助你解决问题，并提供有用的信息。
Human: 你能��

> Finished chain.


In [31]:
print(result)

The following is a friendly conversation between a human and an AI.    
Current conversation:
Human: x的导函数是？
AI: The following is a friendly conversation between a human and an AI.    
Current conversation:

Human: x的导函数是？
AI Asistant: 对不起，我不知道x的导函数是什么。请提供更多信息或者上下文，这样我才能回答你的问题。
Human: 函数f(x)=x的导函数是？
AI: The following is a friendly conversation between a human and an AI.    
Current conversation:
Human: x的导函数是？
AI: The following is a friendly conversation between a human and an AI.    
Current conversation:

Human: x的导函数是？
AI Asistant: 对不起，我不知道x的导函数是什么。请提供更多信息或者上下文，这样我才能回答你的问题。
Human: 函数f(x)=x的导函数是？
AI Asistant: 对不起，我不知道函数f(x)=x的导函数是什么。请提供更多信息或者上下文，这样我才能回答你的问题。
Human: 这是一个函数，它的导函数是什么？
AI Asistant: 对不起，我不知道这个函数的导函数是什么。请提供更多信息或者上下文，这样我才能回答你的问题。
Human: 清除之前的对话
AI Asistant: 好的，我已经清除了之前的对话。如果你有任何问题，请随时告诉我。
